In [2]:
using ReachabilityAnalysis
using Plots
using ControlSystemsBase
using OffsetArrays
using LinearAlgebra
using Polyhedra
using JLD2

In [3]:
"""
	reach(A, x0, W, H; max_order=Inf, reduced_order=2, remove_redundant=true)

Compute reachable sets for the dynamics ``x[k+1] = A x[k] + w``, where ``w`` is a noise term bounded by `W`.  The initial state is `x0`, and the time horizon is `H`.

If `max_order` is given, we reduce order of the reachable set to `reduced_order` when it exceeds this limit.  If `remove_redundant` is true, redundant generators are removed at each step.
"""
function reach(A::AbstractMatrix, x0::AbstractZonotope, W::AbstractZonotope, H::Integer; max_order::Real=Inf, reduced_order::Real=2, remove_redundant::Bool=true)
	# Preallocate x vector
	x = OffsetArray(fill(x0, H+1), OffsetArrays.Origin(0))

	for k = 1:H
		x[k] = minkowski_sum(linear_map(A, x[k-1]), W)
		if remove_redundant
			x[k] = remove_redundant_generators(x[k])
		end
		if order(x[k]) > max_order
			x[k] = reduce_order(x[k], reduced_order)
		end
	end
	
	F = Flowpipe([ReachSet(x_k, k) for (k, x_k) in enumerate(x)])
end

reach

In [6]:
# Accuracy (top-1/top-5) in percentage, parameter in millions, FLOP in billions
efficient_net_surfaces = (
    B0 = (acc1 = 77.1, acc5 = 93.3, para = 5.3, flop = 0.39),
    B1 = (acc1 = 79.1, acc5 = 94.4, para = 7.8, flop = 0.70),
    B2 = (acc1 = 80.1, acc5 = 94.9, para = 9.2, flop = 1.0),
    B3 = (acc1 = 81.6, acc5 = 95.7, para = 12, flop = 1.8),
    B4 = (acc1 = 82.9, acc5 = 96.4, para = 19, flop = 4.2),
    B5 = (acc1 = 83.6, acc5 = 96.7, para = 30, flop = 9.9),
    B6 = (acc1 = 84.0, acc5 = 96.8, para = 43, flop = 19),
    B7 = (acc1 = 84.3, acc5 = 97.0, para = 66, flop = 37),
)

efficient_net_map_full = vcat(([100/b.acc1 - 1;; 100/b.acc5 - 1;; b.flop] for b in efficient_net_surfaces)...)

8×3 Matrix{Float64}:
 0.297017  0.0718114   0.39
 0.264223  0.059322    0.7
 0.248439  0.0537408   1.0
 0.22549   0.0449321   1.8
 0.206273  0.0373444   4.2
 0.196172  0.0341262   9.9
 0.190476  0.0330579  19.0
 0.18624   0.0309278  37.0

## Exhaustive Search

In [ ]:
points(map, Φ) = let
	points = [fill(0, size(Φ, 1)); Inf; Inf]
	for indices in Iterators.product([axes(efficient_net_map, 1) for _ in axes(Φ, 1)]...)
		W = Zonotope(zeros(axes(Φ, 1)), diagm(efficient_net_map[collect(indices), 2]))
		r = reach(Φ, x0, W, 100)
		md = maximum([diameter(x.X) for x in r])
		points = hcat(points, [indices; md; sum(efficient_net_map[collect(indices), 3])])
	end
	points
end